# [NAPARI](https://napari.org/) testing
## and adding [cellpose](https://cellpose-napari.readthedocs.io/en/latest/index.html) (installer [here](https://pypi.org/project/cellpose-napari/0.1.3/))

To upgrade run in a cell:
```python
!pip install cellpose-napari==0.1.3
!pip install cellpose --upgrade
```

```bash
napari -w cellpose-napari
```

### Basic usage:
```python
viewer = napari.view_image(data.astronaut(), rgb=True)
nbscreenshot(viewer)
viewer.close()
```

example notebooks [here](https://github.com/sofroniewn/napari-training-course/blob/master/lessons/)

In [ ]:
## to install
# !pip install napari[all]
# !pip install QT
# !pip install napari[pyqt5]
# !pip install cellpose-napari==0.1.4
# !pip install napari-nikon-nd2
# !pip install magicgui
# !pip install napari-pyclesperanto-assistant
# !pip install napari-clusters-plotter #hdbscan failed
# !pip install napari-plot-profile
# !pip install napari-brightness-contrast
# !pip install napari-curtain
# !pip install napari-3d-ortho-viewer
# !pip install napari-manual-split-and-merge-labels
# !pip install napari-crop
# !pip install napari-stl-exporter

## to upgrade:
# !pip install cellpose-napari==0.1.4
# !pip install cellpose-napari --upgrade
# !pip install cellpose --upgrade
# !pip install napari --upgrade
# !pip install mxnet-mkl

In [ ]:
import napari
import cellpose_napari
import cellpose
from cellpose import models
from napari.utils import nbscreenshot
from tifffile import imread
import numpy as np
from scipy import ndimage
from scipy.stats import mannwhitneyu
import napari_nikon_nd2
import os 
from magicgui import magicgui
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import importlib
import svgutils
from svgutils.compose import *
from plotParams import *

In [ ]:
plotStyle = 'dark';
# plotStyle = 'light';

if plotStyle=='dark':
    # dark background
    params = {"ytick.color" : "w",
              "xtick.color" : "w",
              "axes.labelcolor" : "w",
              "axes.edgecolor" : "w",
             "axes.linewidth" : 3,
             "xtick.major.width" : 3,
             "ytick.major.width" : 3,
             "xtick.major.size" : 8,
             "ytick.major.size" : 8,
             "text.color" : "w"}
    plt.rcParams.update(params)
elif plotStyle=='light':
    # white background
    params = {"ytick.color" : "k",
              "xtick.color" : "k",
              "axes.labelcolor" : "k",
              "axes.edgecolor" : "k",
             "axes.linewidth" : 3,
             "xtick.major.width" : 3,
             "ytick.major.width" : 3,
             "xtick.major.size" : 8,
             "ytick.major.size" : 8,
             "text.color" : "k"}
    plt.rcParams.update(params)
font_prop = font_manager.FontProperties(fname='/System/Library/Fonts/Avenir.ttc')


matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=batlow)

# viewer = napari.Viewer()

print('Opened viewer; plot style is ' + plotStyle)

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.close()

***
# Index <a id='Index'></a>
***
- [Extraction of z planes for analysis](#zExtract)
- [Segmentation with cellpose](#cellSeg)
- [Manual correction of segmentation](#manualCuration)
- [Create thumbnails _WIP_](#thumbnails)
- [Quantification](#quantification)

***
## Extract layers from z-stacks<a id='zExtract'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# getting list of files
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20220107_tbx2bF3s_xOG_s1C"
!ls $dPath/*[002,004].nd2

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2bF3s/xOGs1C/"

# open file

# F3[tbx2b]
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20220107_tbx2bF3s_xOG_s1C/"
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L01_002'; gene = 'wt'; zlims = [58,61];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L02_002'; gene = 'het'; zlims = [45,48];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L03_002'; gene = 'het'; zlims = [43,47];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L04_002'; gene = 'ko'; zlims = [40,47]; # lor
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L05_002'; gene = 'het'; zlims = [64,69];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L06_002'; gene = 'het'; zlims = [38,42]; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L06_004'; gene = 'het'; zlims = [43,46]; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L08_002'; gene = 'het'; zlims = [63,66]; # DAPI is dim
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L09_002'; gene = 'ko'; zlims = [28,31];
# ### fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L09_004'; gene = 'ko'; zlims = [33,39]; # side-view
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L10_002'; gene = 'het'; zlims = [18,25]; # a little curved
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L11_002'; gene = 'het'; zlims = [30,40]; # very curved
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L12_002'; gene = 'wt'; zlims = [18,22];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L13_002'; gene = 'ko'; zlims = [30,34];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L13_004'; gene = 'ko'; zlims = [34,45]; #GFP is dim and variable
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L14_002'; gene = 'het'; zlims = [33,38]; # ONH in FOV
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L15_002'; gene = 'wt'; zlims = [30,34]; # flat and good for DAPI
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L16_002'; gene = 'het'; zlims = [15,18]; # mCherry+ and flat and good for DAPI **EXAMPLE**
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L17_002'; gene = 'wt'; zlims = [28,30]; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L18_002'; gene = 'wt'; zlims = [36,40];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L19_002'; gene = 'ko'; zlims = [30,35]; # not lor!!!!
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L20_002'; gene = 'wt'; zlims = [33,40];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L21_002'; gene = 'het'; zlims = [30,35]; # ONH in FOV
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L22_002'; gene = 'het'; zlims = [32,38];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L23_002'; gene = 'wt'; zlims = [69,71];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L24_002'; gene = 'wt'; zlims = [54,57];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L25_002'; gene = 'ko'; zlims = [43,60]; # not lor!!!!
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L26_002'; gene = 'ko'; zlims = [48,52]; # lor
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L26_004'; gene = 'ko'; zlims = [38,41]; # lor and flat **EXAMPLE**
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L27_002'; gene = 'het'; zlims = [38,41]; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L28_002'; gene = 'het'; zlims = [18,23]; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L29_002'; gene = 'wt'; zlims = [40,45];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L30_002'; gene = 'het'; zlims = [57,59]; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L31_002'; gene = 'wt'; zlims = [72,74]; # flat and good for DAPI
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L32_002'; gene = 'het'; zlims = [40,43];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L33_002'; gene = 'het'; zlims = [27,29]; # ONH in FOV
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L34_002'; gene = 'wt'; zlims = [23,25];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L35_002'; gene = 'ko'; zlims = [56,58]; # lor
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L35_004'; gene = 'ko'; zlims = [27,29];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L36_002'; gene = 'ko'; zlims = [17,19]; # lor
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L36_004'; gene = 'ko'; zlims = [45,47];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L37_002'; gene = 'het'; zlims = [53,55];
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L38_002'; gene = 'het'; zlims = [36,40]; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L39_002'; gene = 'ko'; zlims = [53,74]; # Very very curved; will need manual correction
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L40_002'; gene = 'wt'; zlims = [55,59]; # mCherry+; ONH in FOV


# create subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
if (os.path.isdir(dOut)==False):
    os.mkdir(dOut)
    print('Created output directory')

print('Canvas clear')

In [ ]:
# load whole stack to figure out best planes for cellpose
viewer.open((dPath+fPath + ".nd2"))
viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();
if len(viewer.layers)==2: #no DAPI
    nChannels = 2
    viewer.layers[0].colormap = 'green'
    viewer.layers[0].name = 'R'
    viewer.layers[1].colormap = 'magenta'
    viewer.layers[1].name = 'U'
elif len(viewer.layers)==3: #DAPI
    nChannels = 3
    viewer.layers[0].colormap = 'gray'
    viewer.layers[0].name = 'N'
    viewer.layers[0].opacity = 0.33
    viewer.layers[1].colormap = 'green'
    viewer.layers[1].name = 'R'
    viewer.layers[2].colormap = 'magenta'
    viewer.layers[2].name = 'U'

print('Loaded: ' + fPath)

In [ ]:
# make mips (and remove any previous ones)
# # code for eyes without immuno
zlims = [55,59];
if len(viewer.layers)>3:
    for l in viewer.layers[3:]:
        viewer.layers.remove(l)

for l in [viewer.layers[0],viewer.layers[1],viewer.layers[2]]:
    l.visible = False
#     viewer.layers.remove(l.name + '_mip')
    viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")
    
    
# # for FPs ------- # for OSs # code for eyes with 1D4 immuno
# zlims = [28,30]; zlims2 = [14,18];
# if len(viewer.layers)>3:
#     for l in viewer.layers[3:]:
#         viewer.layers.remove(l)

# for l in [viewer.layers[0],viewer.layers[1]]:
#     l.visible = False
# #     viewer.layers.remove(l.name + '_mip')
#     viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")
    
# for l in [viewer.layers[2]]:
#     l.visible = False
# #     viewer.layers.remove(l.name + '_mip')
#     viewer.add_image(l.data[zlims2[0]:zlims2[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

In [ ]:
# when things look good, save in folder for batch cellpose analysis
l = viewer.layers['R_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['U_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['N_mip']; l.save(dOut + l.name + '.tiff')
print('MIP layers saved for ' + fPath)
# l = viewer.layers['LWS_mip']; l.save(dOut + l.name + '.tiff')

### Go back and run next one &uarr;

In [ ]:
# after running cellpose, rename cellpose images and save before manual curation:
baseName = 'C'
lname = baseName + '_mip'
viewer.layers[lname + '_cp_masks_000'].name = baseName + '_seg'
viewer.layers[lname + '_cp_outlines_000'].name = baseName + '_outlines'
viewer.layers[lname + '_cp_cellprob_000'].name = baseName + '_prob'
viewer.layers[lname + '_cp_flows_000'].name = baseName + '_flows'

l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '.tiff')
l = viewer.layers[baseName + '_outlines']; l.save(dOut + l.name + '.tiff')
l = viewer.layers[baseName + '_prob']; l.save(dOut + l.name + '.tiff')
l = viewer.layers[baseName + '_flows']; l.save(dOut + l.name + '.tiff')

***
## Segmentation using cellpose<a id='cellSeg'>∮</a>
***
[Back to Index](#Index)

## Run cellpose in napari GUI and identify good parameters.
Usually for:
1. xOPS:GFP: diameter = 20; flow_threshold = 0.4 and mask_threshold = 0.0
1. sws1:mCherry: diameter = 40; flow_threshold = 0.4 and mask_threshold = 0.0
1. sws2:mCherry:
1. mws2:GFP:
1. thrb:tdTomato:
    

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2bF3s/xOGs1C/"

# open file

# F3[tbx2b]
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20220107_tbx2bF3s_xOG_s1C/"
fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L01_002'; gene = 'wt'; zlims = [58,61];

# redefine analysis subdirectory
dOut = dAnalysis + gene + '/' + fPath + '/'
print('Viewer cleared...')

# load mips
viewer.open(dOut + 'R' + "_mip.tiff", plugin='builtins', colormap = 'green', blending='additive');
viewer.open(dOut + 'U' + "_mip.tiff", plugin='builtins', colormap = 'magenta', blending='additive');
viewer.open(dOut + 'N' + "_mip.tiff", plugin='builtins', colormap = 'gray', opacity= 0.33, blending='additive');

print('Loaded mips')

## Cellpose: batch analysis
#### Define parameters and files

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

# define analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2bF3s/xOGs1C/"

# collect file paths

gene = 'tbx2b'
fPaths = [
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L01_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L02_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L03_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L04_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L05_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L06_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L06_004',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L08_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L09_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L10_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L11_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L12_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L13_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L13_004',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L14_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L15_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L16_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L17_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L18_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L19_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L20_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L21_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L22_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L23_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L24_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L25_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L26_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L26_004',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L27_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L28_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L29_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L30_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L31_002',
#    '20220107_5dpf_xOG_s1C_tbx2bF3_L32_002',
    '20220107_5dpf_xOG_s1C_tbx2bF3_L33_002',
    '20220107_5dpf_xOG_s1C_tbx2bF3_L34_002',
    '20220107_5dpf_xOG_s1C_tbx2bF3_L35_002',
    '20220107_5dpf_xOG_s1C_tbx2bF3_L35_004',
    '20220107_5dpf_xOG_s1C_tbx2bF3_L36_002',
    '20220107_5dpf_xOG_s1C_tbx2bF3_L36_004',
    '20220107_5dpf_xOG_s1C_tbx2bF3_L37_002',
    '20220107_5dpf_xOG_s1C_tbx2bF3_L38_002',
    '20220107_5dpf_xOG_s1C_tbx2bF3_L39_002',
    '20220107_5dpf_xOG_s1C_tbx2bF3_L40_002',
         ]

# define cellpose params
cpParams = {
    'model' : 'cyto2', # default is 'cyto' or 'cyto2'
    'net_avg' : True,
    'channels' : [0,0], #single channel without nucleus info
    'diameterR' : 20,
    'diameterU' : 40,
    'flow_threshold' : 0.4,
    'mask_threshold' : 0.0
}

# define model to use (e.g. 'cyto2')
model = models.Cellpose(gpu=False, model_type=cpParams['model'])

In [ ]:
print('Starting analysis:')
for fPath in fPaths:
    print('\n' + fPath)
    print('\t Rods')
    dOut = dAnalysis + gene + '/' + fPath + '/' + 'R_mip.tiff'
    img = cellpose.io.imread(dOut)
#     imgplot = plt.imshow(img)
    masks, flows, styles, diams = model.eval(img, 
                                             diameter=cpParams['diameterR'], channels=cpParams['channels'],
                                             do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                             flow_threshold = cpParams['flow_threshold'], mask_threshold = cpParams['mask_threshold'])
    cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
    
#     print('\t UV cones')
#     dOut = dAnalysis + gene + '/' + fPath + '/' + 'U_mip.tiff'
#     img = cellpose.io.imread(dOut)
# #     imgplot = plt.imshow(img)
#     masks, flows, styles, diams = model.eval(img, 
#                                              diameter=cpParams['diameterU'], channels=cpParams['channels'],
#                                              do_3D=False, net_avg = cpParams['net_avg'], interp = True,
#                                              flow_threshold = cpParams['flow_threshold'], mask_threshold = cpParams['mask_threshold'])
#     cellpose.io.masks_flows_to_seg(img, masks, flows, diams, dOut, cpParams['channels']) # save results
    
print('Finished cellpose batch analysis')

***
## Manual correction of cellpose segmentation<a id='manualCuration'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()
# for l in viewer.layers:
#         viewer.layers.remove(l)

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2bF3s/xOGs1C/"

# open file

# F3[tbx2b]
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20220107_tbx2bF3s_xOG_s1C/"
# ALL THESE DONE FOR RODS ONLY
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L01_002'; gene = 'wt';
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L02_002'; gene = 'het';
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L03_002'; gene = 'het';
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L04_002'; gene = 'ko'; # lor
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L05_002'; gene = 'het'; # ok DAPI
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L06_002'; gene = 'het'; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L06_004'; gene = 'het'; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L08_002'; gene = 'het'; # mCherry+; DAPI is dim
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L09_002'; gene = 'ko'; # lor; DAPI is ok
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L10_002'; gene = 'het'; # a little curved
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L11_002'; gene = 'het'; # very curved
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L12_002'; gene = 'wt';
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L13_002'; gene = 'ko'; # lor; DAPI is ok
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L13_004'; gene = 'ko'; # lor; GFP is dim and variable
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L14_002'; gene = 'het'; # ONH in FOV; consider rejecting from analysis
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L15_002'; gene = 'wt'; # flat and good for DAPI
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L16_002'; gene = 'het'; # mCherry+ and flat and good for DAPI **EXAMPLE**
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L17_002'; gene = 'wt'; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L18_002'; gene = 'wt';
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L19_002'; gene = 'ko'; # not lor!!!!
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L20_002'; gene = 'wt';
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L21_002'; gene = 'het'; # ONH in FOV
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L22_002'; gene = 'het';
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L23_002'; gene = 'wt';
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L24_002'; gene = 'wt';
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L25_002'; gene = 'ko'; # not lor!!!!
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L26_002'; gene = 'ko'; # lor
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L26_004'; gene = 'ko'; # lor and flat **EXAMPLE**
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L27_002'; gene = 'het'; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L28_002'; gene = 'het'; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L29_002'; gene = 'wt';
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L30_002'; gene = 'het'; # mCherry+
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L31_002'; gene = 'wt'; # flat and good for DAPI
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L32_002'; gene = 'het';
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L33_002'; gene = 'het'; 
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L34_002'; gene = 'wt'; 
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L35_002'; gene = 'ko'; 
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L35_004'; gene = 'ko'; 
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L36_002'; gene = 'ko'; 
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L36_004'; gene = 'ko'; 
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L37_002'; gene = 'het'; 
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L38_002'; gene = 'het'; mCherry+
#### #fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L39_002'; gene = 'ko'; # too curved; unreliable segmentation. Exclude from analysis
# fPath = '20220107_5dpf_xOG_s1C_tbx2bF3_L40_002'; gene = 'wt'; 


# define subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
print('Viewer cleared...')

# clear key binds
@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    ...

# load mips
viewer.open(dOut + 'R' + "_mip.tiff", plugin='builtins', colormap = 'gray', blending='additive');
viewer.open(dOut + 'U' + "_mip.tiff", plugin='builtins', colormap = 'magenta', blending='additive');
viewer.open(dOut + 'N' + "_mip.tiff", plugin='builtins', colormap = 'bop blue', blending='additive');

# load segmentation
segData = np.load(dOut + 'R' + "_mip" + "_seg.npy", allow_pickle=True).item()
viewer.add_labels(segData['masks'], name='R_seg',blending='additive');
viewer.layers['R_seg'].preserve_labels = True;

# segData = np.load(dOut + 'U' + "_mip" + "_seg.npy", allow_pickle=True).item()
# viewer.add_labels(segData['masks'], name='U_seg',blending='additive');
# viewer.layers['U_seg'].visible = False
# viewer.layers['U_seg'].preserve_labels = True;

# viewer.layers['R_seg'].contour = 5
# viewer.layers['U_seg'].contour = 5

print('Loaded  ' + fPath + ' !\n Now fix it and save it!')

#define useful keyboard shortcuts

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = 'R_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')

print('removeLabel on R_seg ("Shift-X")')

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = 'R_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle R_seg preserve_labels ("k")')
        
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = 'U_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle U_mip visibility ("B")')

@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
    lname = 'R_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True
print('toggle R_mip visibility ("N")')

In [ ]:
# load z-stack if needed
# dPath = "/Volumes/zfSSD/LiImaging/A1R/zf/20201030_CRtbx2ab/"
# viewer.open((dPath+fPath + ".nd2"))
# viewer.layers.remove(viewer.layers[fPath+' [2]']) # remove transmitted detector image
# viewer.layers.remove(viewer.layers[fPath+' [3]']) # remove immuno layer
# viewer.layers[fPath].colormap = 'magenta'
# viewer.layers[fPath].name = 'Mz'
# viewer.layers[fPath+' [1]'].colormap = 'green'
# viewer.layers[fPath+' [1]'].visible = False
# viewer.layers[fPath+' [1]'].name = 'Lz'
# viewer.layers.select_next();
# print('Loaded!')

#### resave curated segmentation after napari-ing around (only doing R_mip)

In [ ]:
baseName = 'R'
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + fPath)

### [Go back and run next one &uarr;](#manualCuration)
---

In [ ]:
# manual merging of labels
mergeLabels = [167,168]
# lname = 'M_seg'
lname = 'L_seg'
if viewer.layers[lname].visible == False:
    print('Layer not active')
else:
    tempd = viewer.layers[lname].data
    tempd[tempd == mergeLabels[1]]=mergeLabels[0]
    viewer.layers[lname].data = tempd
    print('Merged in ' + lname)

In [ ]:
# manually delete label
selLabel = 49
lname = 'L_seg'
tempd = viewer.layers[lname].data
tempd[tempd == selLabel]=0
viewer.layers[lname].data = tempd

#### resave curated segmentation after napari-ing around

In [ ]:
# baseName = 'R'
# lname = baseName + '_seg'
# l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated_incomplete.tiff')
# # l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
# viewer.layers['L_mip'].visible = True
# viewer.layers['L_seg'].visible = True
# viewer.layers['M_seg'].contour = 0
# viewer.layers['M_mip'].visible = False
# viewer.layers['M_seg'].visible = False
# print('Done with ' + lname)

# @viewer.bind_key('Shift-x', overwrite=True)
# def removeLabel(viewer):
# #     lname = 'M_seg'mvvvv
#     lname = 'L_seg'
#     tempd = viewer.layers[lname].data
#     tempd[tempd == viewer.layers[lname].selected_label]=0
#     viewer.layers[lname].data = tempd
#     print('Cut!')

# print('removeLabel on L_seg ("Shift-X")')

# @viewer.bind_key('k', overwrite=True)
# def toggle_sel(viewer):
#     lname = 'L_seg'
#     if (viewer.layers[lname].preserve_labels == True):
#         viewer.layers[lname].preserve_labels = False
#     elif (viewer.layers[lname].preserve_labels == False):
#         viewer.layers[lname].preserve_labels = True
# print('toggle L_seg preserve_labels ("k")')

# @viewer.bind_key('b', overwrite=True)
# def toggle_sel(viewer):
#     lname = 'M_mip'
#     if (viewer.layers[lname].visible == True):
#         viewer.layers[lname].visible = False
#     elif (viewer.layers[lname].visible == False):
#         viewer.layers[lname].visible = True
# print('toggle L_mip visibility ("B")')

        
# @viewer.bind_key('n', overwrite=True)
# def toggle_sel(viewer):
#     lname = 'L_mip'
#     if (viewer.layers[lname].visible == True):
#         viewer.layers[lname].visible = False
#     elif (viewer.layers[lname].visible == False):
#         viewer.layers[lname].visible = True

# print('toggle L_mip visibility ("N")')

### Reload curated segmentations after saving

In [ ]:
# remove all except mips
if len(viewer.layers)>2:
    for l in viewer.layers[2:]:
        viewer.layers.remove(l)

In [ ]:
# viewer.open(dOut + 'M' + "_seg_curated_incomplete.tiff", name='M_seg', plugin='builtins');
viewer.open(dOut + 'R' + "_seg_curated.tiff", name='R_seg', plugin='builtins');
viewer.layers['R_seg'].contour = 5

print(fPath)
nR = len(np.unique(viewer.layers['R_seg'].data))-1
print("Image has {0} Rods".format(nR))
# nR = len(np.unique(viewer.layers['L_seg'].data))-1
# print("Image has {0} L cones".format(nR))

In [ ]:
# remove all lauers
if len(viewer.layers)>0:
    for l in viewer.layers[0:]:
        viewer.layers.remove(l)

#### 

***
## To Do: Batch export of png with mips and contours of curated segmentation<a id='thumbnails'>∮</a>
***
[Back to Index](#Index)

This could do the trick:
```python
def blended_img(viewer):
    import napari
    import numpy as np
    
    blended = np.zeros(viewer.layers[0].data.shape + (4,))
    for layer in viewer.layers:
        # normalize data by clims
        normalized_data = (layer.data - layer.contrast_limits[0]) / (
        layer.contrast_limits[1] - layer.contrast_limits[0]
    )
        colormapped_data = layer.colormap.map(normalized_data.flatten())
        colormapped_data = colormapped_data.reshape(normalized_data.shape + (4,))

        blended = blended + colormapped_data
    
    blended[..., 3] = 1 # set alpha channel to 1

    return blended
```

***
## Quantification<a id='quantification'></a>: This needs updated (Feb 2022)
***
[Back to Index](#Index)

### Compile counts by looping through all files

In [ ]:
def cellCounter(viewer,dAnalysis,gene,fPath,photoLabel):
    import os.path
    # clear viewer
    for l in viewer.layers:
        viewer.layers.remove(l)
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    dOut = dAnalysis + gene + '/' + fPath + '/'
    # load segmentation
    segPath = dOut + photoLabel + "_seg_curated.tiff"
    if os.path.isfile(segPath):
        viewer.open(segPath, name='Seg', plugin='builtins', blending='additive');
        nCells = np.unique(viewer.layers['Seg'].data).shape[0]
    else:
        nCells = float("nan")
    print(fPath + ': nR = ' + str(nCells)) 
#     np.savez(dOut + 'quantificationGFP.npz',
#              lcones=lcones,
#              lRFP=lRFP,
#              lGFP=lGFP,
#              lRFPsd=lRFPsd,
#              lGFPsd=lGFPsd,
#              lRtile=lRtile,
#              lGtile=lGtile)
#     results = zip(['lcones','lRFP','lGFP','lRFPsd','lGFPsd','lRtile','lGtile'],
#                  [lcones,lRFP,lGFP,lRFPsd,lGFPsd,lRtile,lGtile])
    
    return nCells

In [ ]:
# because it's simple, just going to create csv manually
# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRskor1aF0s/xOGaCT/"

gene = 'wt'
wt_fPaths = [
    "20211129_5dpf_xOG_aCtdT_wt_L07_002",
    "20211129_5dpf_xOG_aCtdT_wt_L07_004",
    "20211129_5dpf_xOG_aCtdT_wt_L09_002",
    "20211129_5dpf_xOG_aCtdT_wt_L09_004",
         ]
for fPath in wt_fPaths:
    cellCounter(viewer,dAnalysis,gene,fPath,'R')
    
    
gene = 'skor1a'
skor1a_fPaths = [
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L07_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L07_004",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L08_002", #from fragment analysis: this is wt
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L08_004", #from fragment analysis: this is wt
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L09_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L09_004",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L10_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L11_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L12_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L12_004",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L13_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L13_004",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L14_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L15_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L16_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L17_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L17_004",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L18_002",
    "20211129_5dpf_xOG_aCtdT_skor1aF0_L18_004",
         ]
for fPath in skor1a_fPaths:
    cellCounter(viewer,dAnalysis,gene,fPath,'R')
        


In [ ]:
# viewer = napari.Viewer()

In [ ]:
def calcMillie(viewer,lname):
    outHist,outBins = np.histogram(viewer.layers[lname].data.flatten(), bins = 1000)
    return outHist,outBins

def GFPanalyzer(viewer,dAnalysis,gene,fPath, erosionValue=10):
    # clear viewer
    for l in viewer.layers:
        viewer.layers.remove(l)
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    dOut = dAnalysis + gene + '/' + fPath + '/'
    print('Quantifying GFP in L cones of ' + fPath + '(' + gene + ')\n')
    # load mips
    viewer.open(dOut + 'M' + "_mip.tiff", plugin='builtins', colormap = 'green', blending='additive');
    viewer.open(dOut + 'L' + "_mip.tiff", plugin='builtins', colormap = 'magenta', blending='additive');
    # viewer.open(dOut + 'M' + "_seg_curated_incomplete.tiff", name='M_seg', plugin='builtins', blending='additive');
    viewer.open(dOut + 'L' + "_seg_curated.tiff", name='L_seg', plugin='builtins', blending='additive');
    # viewer.layers['M_seg'].preserve_labels = True;
    viewer.layers['L_seg'].preserve_labels = True;
    viewer.layers['L_seg'].contour = 5
    # erode L cone segmentation
    viewer.layers['L_seg'].data = ndimage.grey_erosion(viewer.layers['L_seg'].data, size=(erosionValue,erosionValue))
    # divide images into 1000 bins
    mH, mB = calcMillie(viewer,'M_mip')
    lH, lB = calcMillie(viewer,'L_mip')

    lcones = np.unique(viewer.layers['L_seg'].data)
    lcones = lcones[lcones>0]
    lRFP = np.empty(np.shape(lcones))
    lGFP = np.empty(np.shape(lcones))
    lRFPsd = np.empty(np.shape(lcones))
    lGFPsd = np.empty(np.shape(lcones))
    lRtile = np.empty(np.shape(lcones))
    lGtile = np.empty(np.shape(lcones))
    
    i=0;
    for l in lcones:
        lRFP[i] = np.mean(viewer.layers['L_mip'].data[viewer.layers['L_seg'].data==l])
        lGFP[i] = np.mean(viewer.layers['M_mip'].data[viewer.layers['L_seg'].data==l])
        lRFPsd[i] = np.std(viewer.layers['L_mip'].data[viewer.layers['L_seg'].data==l])
        lGFPsd[i] = np.std(viewer.layers['M_mip'].data[viewer.layers['L_seg'].data==l])
        lRtile[i] = len(lB[lB<lRFP[i]])
        lGtile[i] = len(mB[mB<lGFP[i]])
        i=i+1;
        
    np.savez(dOut + 'quantificationGFP.npz',
             lcones=lcones,
             lRFP=lRFP,
             lGFP=lGFP,
             lRFPsd=lRFPsd,
             lGFPsd=lGFPsd,
             lRtile=lRtile,
             lGtile=lGtile)
    results = zip(['lcones','lRFP','lGFP','lRFPsd','lGFPsd','lRtile','lGtile'],
                 [lcones,lRFP,lGFP,lRFPsd,lGFPsd,lRtile,lGtile])
    
    return results

In [ ]:
erodeV = 10; #12;

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2F0s/m2Gt2T/"

# open file

# uninjected
### 20201009: imaged 3dpf and 5dpf larvae. Analyzing 5 dpf for paper
gene = 'wt'
wt_fPaths = ["20201009_5dpf_Rh22G_trb2T_wt_i09b",
          "20201009_5dpf_Rh22G_trb2T_wt_i09d",
          "20201009_5dpf_Rh22G_trb2T_wt_i10b",
          "20201009_5dpf_Rh22G_trb2T_wt_i10d",
          "20201009_5dpf_Rh22G_trb2T_wt_i11b",
          "20201009_5dpf_Rh22G_trb2T_wt_i11d",
          "20201009_5dpf_Rh22G_trb2T_wt_i12b",
          "20201009_5dpf_Rh22G_trb2T_wt_i12d",
          "20210716_5dpf_m2G_t2T_uninj_L101b",
          "20210716_5dpf_m2G_t2T_uninj_L101d",
          "20210716_5dpf_m2G_t2T_uninj_L102b",
          "20210716_5dpf_m2G_t2T_uninj_L103b",
          "20210716_5dpf_m2G_t2T_uninj_L103d",
          "20210716_5dpf_m2G_t2T_uninj_L104b",
          "20210716_5dpf_m2G_t2T_uninj_L104d",
          "20210716_5dpf_m2G_t2T_uninj_L105b",
          "20210716_5dpf_m2G_t2T_uninj_L106b",
          "20210716_5dpf_m2G_t2T_uninj_L106d",
         ]
for fPath in wt_fPaths:
    GFPanalyzer(viewer,dAnalysis,gene,fPath, erosionValue=erodeV)
    for l in viewer.layers:
        viewer.layers.remove(l)
    
# tbx2a
gene = 'tbx2a';
tbx2a_fPaths = ["20201009_5dpf_Rh22G_trb2T_CRtbx2a_i07b",
#                 "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i07b",  # is this supposed to be d????
                # #### fPath = "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i08b", #very curved stack. would require oblique cut
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i08d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i09b",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i09d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i10b", #done; this could be example
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i10d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i11b", #done; very clean L-cone seg
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i11d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i12b",
                #### fPath = "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i12d", # this in lens up
               ]
for fPath in tbx2a_fPaths:
    GFPanalyzer(viewer,dAnalysis,gene,fPath, erosionValue=erodeV)
    for l in viewer.layers:
        viewer.layers.remove(l)

# tbx2b
gene = 'tbx2b';
tbx2b_fPaths = ["20201030_5dpf_Rh22G_trb2T_CRtbx2b_i07b",
                "20201030_5dpf_Rh22G_trb2T_CRtbx2b_i07d",
                 "20201030_5dpf_Rh22G_trb2T_CRtbx2b_i08b",
                "20201030_5dpf_Rh22G_trb2T_CRtbx2b_i08d",
                "20210716_5dpf_m2G_t2T_tbx2b_L102b",
                "20210716_5dpf_m2G_t2T_tbx2b_L102d",
                "20210716_5dpf_m2G_t2T_tbx2b_L104b",
                "20210716_5dpf_m2G_t2T_tbx2b_L104d",
                "20210716_5dpf_m2G_t2T_tbx2b_L105b",
                "20210716_5dpf_m2G_t2T_tbx2b_L105d",
                "20210716_5dpf_m2G_t2T_tbx2b_L106b",
                "20210716_5dpf_m2G_t2T_tbx2b_L107b",
                "20210716_5dpf_m2G_t2T_tbx2b_L108b",
                "20210716_5dpf_m2G_t2T_tbx2b_L108d",
                "20210716_5dpf_m2G_t2T_tbx2b_L109b",
                "20210716_5dpf_m2G_t2T_tbx2b_L110b",
                "20210716_5dpf_m2G_t2T_tbx2b_L111b",
               ]

for fPath in tbx2b_fPaths:
    GFPanalyzer(viewer,dAnalysis,gene,fPath, erosionValue=erodeV)
    for l in viewer.layers:
        viewer.layers.remove(l)

print('Finished analysis')






In [ ]:
for l in viewer.layers:
    print(l)
    viewer.layers.remove(l)

In [ ]:
# viewer = napari.Viewer()

In [ ]:
# collect all results

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2F0s/m2Gt2T/"

# open file

# uninjected
### 20201009: imaged 3dpf and 5dpf larvae. Analyzing 5 dpf for paper
gene = 'wt'
wt_fPaths = ["20201009_5dpf_Rh22G_trb2T_wt_i09b",
          "20201009_5dpf_Rh22G_trb2T_wt_i09d",
          "20201009_5dpf_Rh22G_trb2T_wt_i10b",
          "20201009_5dpf_Rh22G_trb2T_wt_i10d",
          "20201009_5dpf_Rh22G_trb2T_wt_i11b",
          "20201009_5dpf_Rh22G_trb2T_wt_i11d",
          "20201009_5dpf_Rh22G_trb2T_wt_i12b",
          "20201009_5dpf_Rh22G_trb2T_wt_i12d",
          "20210716_5dpf_m2G_t2T_uninj_L101b",
          "20210716_5dpf_m2G_t2T_uninj_L101d",
          "20210716_5dpf_m2G_t2T_uninj_L102b",
          "20210716_5dpf_m2G_t2T_uninj_L103b",
          "20210716_5dpf_m2G_t2T_uninj_L103d",
          "20210716_5dpf_m2G_t2T_uninj_L104b",
          "20210716_5dpf_m2G_t2T_uninj_L104d",
          "20210716_5dpf_m2G_t2T_uninj_L105b",
          "20210716_5dpf_m2G_t2T_uninj_L106b",
          "20210716_5dpf_m2G_t2T_uninj_L106d",
         ]
# tbx2a
gene = 'tbx2a';
tbx2a_fPaths = ["20201009_5dpf_Rh22G_trb2T_CRtbx2a_i07b",
#                 "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i07b",  # is this supposed to be d????
                # #### fPath = "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i08b", #very curved stack. would require oblique cut
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i08d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i09b",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i09d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i10b", #done; this could be example
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i10d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i11b", #done; very clean L-cone seg
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i11d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i12b",
                #### fPath = "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i12d", # this in lens up
               ]
# tbx2b
gene = 'tbx2b';
tbx2b_fPaths = ["20201030_5dpf_Rh22G_trb2T_CRtbx2b_i07b",
                "20201030_5dpf_Rh22G_trb2T_CRtbx2b_i07d",
                "20201030_5dpf_Rh22G_trb2T_CRtbx2b_i08b",
                "20201030_5dpf_Rh22G_trb2T_CRtbx2b_i08d",
                "20210716_5dpf_m2G_t2T_tbx2b_L102b",
                "20210716_5dpf_m2G_t2T_tbx2b_L102d",
                "20210716_5dpf_m2G_t2T_tbx2b_L104b",
                "20210716_5dpf_m2G_t2T_tbx2b_L104d",
                "20210716_5dpf_m2G_t2T_tbx2b_L105b",
                "20210716_5dpf_m2G_t2T_tbx2b_L105d",
                "20210716_5dpf_m2G_t2T_tbx2b_L106b",
                "20210716_5dpf_m2G_t2T_tbx2b_L107b",
                "20210716_5dpf_m2G_t2T_tbx2b_L108b",
                "20210716_5dpf_m2G_t2T_tbx2b_L108d",
                "20210716_5dpf_m2G_t2T_tbx2b_L109b",
                "20210716_5dpf_m2G_t2T_tbx2b_L110b",
                "20210716_5dpf_m2G_t2T_tbx2b_L111b",
               ]
                
                
rName = 'lGtile'

wtResults = []
i=1;
for fPath in wt_fPaths:
    gene = 'wt'
    dOut = dAnalysis + gene + '/' + fPath + '/'
    results = np.load(dOut + 'quantificationGFP.npz')
    wtResults.append(results[rName])
    i=i+1;

aResults = []
i=1;
for fPath in tbx2a_fPaths:
    gene = 'tbx2a'
    dOut = dAnalysis + gene + '/' + fPath + '/'
    results = np.load(dOut + 'quantificationGFP.npz')
    aResults.append(results[rName])
    i=i+1;

bResults = []
i=1;
for fPath in tbx2b_fPaths:
    gene = 'tbx2b'
    dOut = dAnalysis + gene + '/' + fPath + '/'
    results = np.load(dOut + 'quantificationGFP.npz')
    bResults.append(results[rName])
    i=i+1;


# combine across images
wtResults = np.concatenate(wtResults).ravel()
aResults = np.concatenate(aResults).ravel()
bResults = np.concatenate(bResults).ravel()

# plot histogram
pdfV = True
# wtH, wtB = np.histogram(wtResults, bins = 50, density=pdfV)
# aH, aB = np.histogram(aResults, bins = 50, density=pdfV)
# bH, bB = np.histogram(bResults, bins = 50, density=pdfV)

# plot histogram but first calculate a fixed bin number for all genotypes
# allBins = np.histogram_bin_edges(np.concatenate([wtResults,aResults,bResults]).ravel(),bins='fd');
allBins = np.arange(0,1010,10);
allBinsC = (allBins[:-1] + allBins[1:]) / 2 # Centers
wtH, wtB = np.histogram(wtResults, bins = allBins, density=pdfV)
aH, aB = np.histogram(aResults, bins = allBins, density=pdfV)
bH, bB = np.histogram(bResults, bins = allBins, density=pdfV)

# # this can be used for cumulative distribution then normalized
# wtH = np.cumsum(wtH)/np.max(np.cumsum(wtH));
# aH = np.cumsum(aH)/np.max(np.cumsum(aH));
# bH = np.cumsum(bH)/np.max(np.cumsum(bH));

wt95CI = np.min(allBinsC[(np.cumsum(wtH))>.095])/1000;
wGBright = np.sum(wtH[allBinsC>wt95CI*1000]);
aGBright = np.sum(aH[allBinsC>wt95CI*1000]);
bGBright = np.sum(bH[allBinsC>wt95CI*1000]);

print('GFP threshold is {0}'.format(wt95CI))

baseColor = 'k'
if (plotStyle=='light'):
    baseColor = 'k';

# fH, (axH1,axH2,axH3) = plt.subplots(nrows=3, ncols=1, figsize=(10, 20))
fH, (axH1,axH2,axH3) = plt.subplots(nrows=1, ncols=3, figsize=(30, 8))
axH1.stairs(wtH,wtB/1000, fill=True, color = baseColor, alpha = 1.0)
axH2.stairs(wtH,wtB/1000, fill=True, color = baseColor, alpha = 1.0)
axH3.stairs(wtH,wtB/1000, fill=True, color = baseColor, alpha = 1.0)
axH2.stairs(aH,aB/1000, fill=True, color = zfC['lws1'], alpha = 0.8)
axH3.stairs(bH,bB/1000, fill=True, color = zfC['sws2'], alpha = 0.8)


axH1.plot([wt95CI,wt95CI],[0,0.016],color=baseColor)
axH2.plot([wt95CI,wt95CI],[0,0.016],color=baseColor)
axH3.plot([wt95CI,wt95CI],[0,0.016],color=baseColor)


# axH.set_title('Distribution of GFP inside L cones')
for axH in (axH1,axH2,axH3):
    formatFigureMain(fH, axH, _)
    axH.set_xlabel('Normalized GFP values')
    if pdfV:
        axH.set_ylabel('Probability density')
    else:
        axH.set_ylabel('Counts')
## normalized
# axH.stairs(wtH/np.max(wtH),wtB, fill=True, color = 'k', alpha = 0.5)
# axH.stairs(aH/np.max(aH),aB, fill=True, color = 'r', alpha = 0.5)
# axH.stairs(bH/np.max(bH),bB, fill=True, color = 'b', alpha = 0.5)

dPath = '/Users/angueyraaristjm/Documents/LiLab/Manuscripts/2021_zfConeRNAseq/resources/'
# dPath = '/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/'
plt.savefig(dPath + "CRtbx2F0_LM_hist.svg", transparent=True, format="svg")

print('# cells of bright GFP cells: wt={0}, tbx2a={1}, tbx2b={2}'.format(wGBright*1000,aGBright*1000,bGBright*1000))

In [ ]:

# fH, (axH1,axH2,axH3) = plt.subplots(nrows=3, ncols=1, figsize=(10, 20))
fH, axH = plt.subplots(nrows=1, ncols=1, figsize=(9, 6))


# plot histogram
pdfV = True
# wtH, wtB = np.histogram(wtResults, bins = 50, density=pdfV)
# aH, aB = np.histogram(aResults, bins = 50, density=pdfV)
# bH, bB = np.histogram(bResults, bins = 50, density=pdfV)

# plot histogram but first calculate a fixed bin number for all genotypes
# allBins = np.histogram_bin_edges(np.concatenate([wtResults,aResults,bResults]).ravel(),bins='fd');
allBins = np.arange(0,1010,10);
allBinsC = (allBins[:-1] + allBins[1:]) / 2 # Centers
wtH, wtB = np.histogram(wtResults, bins = allBins, density=pdfV)
aH, aB = np.histogram(aResults, bins = allBins, density=pdfV)
bH, bB = np.histogram(bResults, bins = allBins, density=pdfV)

# # this can be used for cumulative distribution then normalized
# wtH = np.cumsum(wtH)/np.max(np.cumsum(wtH));
# aH = np.cumsum(aH)/np.max(np.cumsum(aH));
# bH = np.cumsum(bH)/np.max(np.cumsum(bH));

wt95CI = np.min(allBinsC[(np.cumsum(wtH))>.095])/1000;
wGBright = np.sum(wtH[allBinsC>wt95CI*1000]);
aGBright = np.sum(aH[allBinsC>wt95CI*1000]);
bGBright = np.sum(bH[allBinsC>wt95CI*1000]);


axH.stairs(wtH,wtB/1000, fill=True, color = 'k', alpha = .5)
axH.stairs(aH,aB/1000, fill=True, color = zfC['lws1'], alpha = .5)
axH.stairs(bH,bB/1000, fill=True, color = zfC['sws2'], alpha = .5)


axH.plot([wt95CI,wt95CI],[0,0.016],color=baseColor)

formatFigureMain(fH, axH, _)
# axH.set_title('Distribution of GFP inside L cones')
axH.set_xlabel('Normalized GFP values')
if pdfV:
    axH.set_ylabel('Probability density')
else:
    axH.set_ylabel('Counts')
## normalized
# axH.stairs(wtH/np.max(wtH),wtB, fill=True, color = 'k', alpha = 0.5)
# axH.stairs(aH/np.max(aH),aB, fill=True, color = 'r', alpha = 0.5)
# axH.stairs(bH/np.max(bH),bB, fill=True, color = 'b', alpha = 0.5)

# dPath = '/Users/angueyraaristjm/Documents/LiLab/Manuscripts/2021_zfConeRNAseq/resources/'
dPath = '/Users/angueyraaristjm/Documents/LiLab/Conferences/2021_RCS/resources/'
# plt.savefig(dPath + "CRtbx2F0_LM_hist.svg", transparent=True, format="svg")

print('# cells of bright GFP cells: wt={0}, tbx2a={1}, tbx2b={2}'.format(wGBright*1000,aGBright*1000,bGBright*1000))

### After establishing threshold based on all data, count number of cells exceeding threshold for each eye

In [ ]:
# collect all results

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2F0s/m2Gt2T/"

# open file

# uninjected
### 20201009: imaged 3dpf and 5dpf larvae. Analyzing 5 dpf for paper
gene = 'wt'
wt_fPaths = ["20201009_5dpf_Rh22G_trb2T_wt_i09b",
          "20201009_5dpf_Rh22G_trb2T_wt_i09d",
          "20201009_5dpf_Rh22G_trb2T_wt_i10b",
          "20201009_5dpf_Rh22G_trb2T_wt_i10d",
          "20201009_5dpf_Rh22G_trb2T_wt_i11b",
          "20201009_5dpf_Rh22G_trb2T_wt_i11d",
          "20201009_5dpf_Rh22G_trb2T_wt_i12b",
          "20201009_5dpf_Rh22G_trb2T_wt_i12d",
          "20210716_5dpf_m2G_t2T_uninj_L101b",
          "20210716_5dpf_m2G_t2T_uninj_L101d",
          "20210716_5dpf_m2G_t2T_uninj_L102b",
          "20210716_5dpf_m2G_t2T_uninj_L103b",
          "20210716_5dpf_m2G_t2T_uninj_L103d",
          "20210716_5dpf_m2G_t2T_uninj_L104b",
          "20210716_5dpf_m2G_t2T_uninj_L104d",
          "20210716_5dpf_m2G_t2T_uninj_L105b",
          "20210716_5dpf_m2G_t2T_uninj_L106b",
          "20210716_5dpf_m2G_t2T_uninj_L106d",
         ]
# tbx2a
gene = 'tbx2a';
tbx2a_fPaths = ["20201009_5dpf_Rh22G_trb2T_CRtbx2a_i07b",
#                 "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i07b",  # is this supposed to be d????
                # #### fPath = "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i08b", #very curved stack. would require oblique cut
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i08d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i09b",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i09d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i10b", #done; this could be example
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i10d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i11b", #done; very clean L-cone seg
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i11d",
                "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i12b",
                #### fPath = "20201009_5dpf_Rh22G_trb2T_CRtbx2a_i12d", # this in lens up
               ]
# tbx2b
gene = 'tbx2b';
tbx2b_fPaths = ["20201030_5dpf_Rh22G_trb2T_CRtbx2b_i07b",
                "20201030_5dpf_Rh22G_trb2T_CRtbx2b_i07d",
                "20201030_5dpf_Rh22G_trb2T_CRtbx2b_i08b",
                "20201030_5dpf_Rh22G_trb2T_CRtbx2b_i08d",
                "20210716_5dpf_m2G_t2T_tbx2b_L102b",
                "20210716_5dpf_m2G_t2T_tbx2b_L102d",
                "20210716_5dpf_m2G_t2T_tbx2b_L104b",
                "20210716_5dpf_m2G_t2T_tbx2b_L104d",
                "20210716_5dpf_m2G_t2T_tbx2b_L105b",
                "20210716_5dpf_m2G_t2T_tbx2b_L105d",
                "20210716_5dpf_m2G_t2T_tbx2b_L106b",
                "20210716_5dpf_m2G_t2T_tbx2b_L107b",
                "20210716_5dpf_m2G_t2T_tbx2b_L108b",
                "20210716_5dpf_m2G_t2T_tbx2b_L108d",
                "20210716_5dpf_m2G_t2T_tbx2b_L109b",
                "20210716_5dpf_m2G_t2T_tbx2b_L110b",
                "20210716_5dpf_m2G_t2T_tbx2b_L111b",
               ]
                
rName = 'lGtile'

wtResults = []
i=1;
for fPath in wt_fPaths:
    gene = 'wt'
    dOut = dAnalysis + gene + '/' + fPath + '/'
    results = np.load(dOut + 'quantificationGFP.npz')
#     wtResults.append(np.count_nonzero(results['lGtile']>wt95CI*1000)) #this is for absolute counts
    wtResults.append(np.divide(np.count_nonzero(results['lGtile']>wt95CI*1000),len(results['lGtile']))) #this is for fraction
    i=i+1;

    
aResults = []
i=1;
for fPath in tbx2a_fPaths:
    gene = 'tbx2a'
    dOut = dAnalysis + gene + '/' + fPath + '/'
    results = np.load(dOut + 'quantificationGFP.npz')
#     aResults.append(np.count_nonzero(results['lGtile']>wt95CI*1000)) #this is for absolute counts
    aResults.append(np.divide(np.count_nonzero(results['lGtile']>wt95CI*1000),len(results['lGtile']))) #this is for fraction
    i=i+1;

bResults = []
i=1;
for fPath in tbx2b_fPaths:
    gene = 'tbx2b'
    dOut = dAnalysis + gene + '/' + fPath + '/'
    results = np.load(dOut + 'quantificationGFP.npz')
#     bResults.append(np.count_nonzero(results['lGtile']>wt95CI*1000)) #this is for absolute counts
    bResults.append(np.divide(np.count_nonzero(results['lGtile']>wt95CI*1000),len(results['lGtile']))) #this is for fraction
    i=i+1;

geneList = ['wt','tbx2a','tbx2b']


plotname = ''
fH, axH = plt.subplots(figsize= [6,8])

barW = np.size(geneList)+2; # bar width
barD = np.size(geneList)+1; # bar distance whitin 1 photoreceptor subtype
barP = -np.size(geneList)+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


jitter = estimateJitter(wtResults)/400
pH = plt.bar([barStep+1/barD], np.mean(wtResults), width=1/barW, color=zfC['lws1']);
pH = plt.errorbar([barStep+1/barD], np.mean(wtResults), yerr = [[0.025],[np.std(wtResults)]], ecolor=lighten_color(zfC['lws1'],1), elinewidth=3, capsize=15, capthick=3, zorder=7)
pH = plt.scatter(np.ones(len(wtResults))*[barStep+1/barD]+jitter, wtResults, color=lighten_color(zfC['lws1'],.75), zorder=8, marker = zfGm['wt'], s=100, edgecolor='#000000', linewidth=0.5, alpha = .6);
barPos = np.append(barPos,(barStep+1/barD))



jitter = estimateJitter(aResults)/400
pH = plt.bar([barStep+2/barD], np.mean(aResults), width=1/barW, color=zfC['lws1']);
pH = plt.errorbar([barStep+2/barD], np.mean(aResults), yerr = [[0.025],[np.std(aResults)]], ecolor=lighten_color(zfC['lws1'],1), elinewidth=3, capsize=15, capthick=3, zorder=7)
pH = plt.scatter(np.ones(len(aResults))*[barStep+2/barD]+jitter, aResults, color=lighten_color(zfC['lws1'],.75), zorder=8, marker = zfGm['tbx2a'], s=100, edgecolor='#000000', linewidth=0.5, alpha = .6);
barPos = np.append(barPos,(barStep+2/barD))

jitter = estimateJitter(bResults)/400
pH = plt.bar([barStep+3/barD], np.mean(bResults), width=1/barW, color=zfC['lws1']);
pH = plt.errorbar([barStep+3/barD], np.mean(bResults), yerr = [[0.025],[np.std(bResults)]], ecolor=lighten_color(zfC['lws1'],1), elinewidth=3, capsize=15, capthick=3, zorder=7)
pH = plt.scatter(np.ones(len(bResults))*[barStep+3/barD]+jitter, bResults, color=lighten_color(zfC['lws1'],.75), zorder=8, marker = zfGm['tbx2b'], s=100, edgecolor='#000000', linewidth=0.5, alpha = .6);
barPos = np.append(barPos,(barStep+3/barD))

formatFigureMain(fH, axH, pH)

# axH.set_ylabel('GFP+ L cones per 64 x 64 $\mu$m$^2$', fontproperties=formatFigureMain(fH, axH, pH))
# axH.set_ylim([0,180]);
# stath = 170

axH.set_ylabel('Fraction of GFP+ L cones', fontproperties=formatFigureMain(fH, axH, pH))
axH.set_ylim([0,1]);
stath = 0.85

geneList = ['control','F$\emptyset$[tbx2a]','F$\emptyset$[tbx2b]']

axH.xaxis.set_tick_params(rotation=45)
axH.set_xticks(np.sort(barPos));
axH.set_xticklabels(geneList);

hH = plt.hlines(stath, barPos[0], barPos[1], colors='#000000',linewidth=4)
plt.text((barPos[0]+barPos[1])/2, stath, '*', font_properties=font_prop, fontsize=36, ha='center')# plt.savefig(savePath + "Counts_tbx2_RU.svg", transparent=True, format="svg")


dPath = '/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/Manuscripts/2021_zfConeRNAseq/resources/'
# dPath = '/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/'
plt.savefig(dPath + "CRtbx2F0_GFPLcones.svg", transparent=True, format="svg")



In [ ]:
print('wt fractions:')
print('\tmean = {0:.3f}%, s.d. = {1:.5f}%'.format(np.mean(wtResults)*100,np.std(wtResults)*100))
print('tbx2a fractions:')
print('\tmean = {0:.3f}%, s.d. = {1:.5f}%'.format(np.mean(aResults)*100,np.std(aResults)*100))
print('tbx2b fractions:')
print('\tmean = {0:.3f}%, s.d. = {1:.5f}%'.format(np.mean(bResults)*100,np.std(bResults)*100))
# Mann Whitney 
Au, pA = mannwhitneyu(wtResults, aResults)
print('wt vs. tbx2a:')
print('\tU = {0:.3f}, p = {1:.5f}'.format(Au,pA))
Bu, pB = mannwhitneyu(wtResults, bResults)
print('wt vs. tbx2b:')
print('\tU = {0:.3f}, p = {1:.5f}'.format(Bu,pB))

In [ ]:
# divide images into 1000 bins
mH, mB = np.histogram(viewer.layers['M_mip'].data.flatten(), bins = 1000)
lH, lB = np.histogram(viewer.layers['L_mip'].data.flatten(), bins = 1000)


fH, axH = plt.subplots(figsize=(8,8))
axH.stairs(lH,lB, fill=True, color = 'm', alpha = 0.5)
axH.stairs(mH,mB, fill=True, color = 'g', alpha = 0.5)

In [ ]:
viewer.close()

In [ ]:
viewer

```python
# change keybind for default actions:
@viewer.bind_key('F1', overwrite=True)
def eraserMode(viewer):
    viewer.layers.selection.active.mode = 'fill'
    viewer.layers.selection.active.mode = 'paint'
    viewer.layers.selection.active.mode = 'erase'
    viewer.layers.selection.active.mode = 'pick'
    viewer.layers.selection.active.mode = 'pan_zoom'

# napari native functions for other actions

@register_label_action(
    trans._(
        "Set the currently selected label to the largest used label plus one."
    ),
)
def new_label(layer):
    """Set the currently selected label to the largest used label plus one."""
    layer.selected_label = layer.data.max() + 1


@register_label_action(
    trans._("Decrease the currently selected label by one."),
)
def decrease_label_id(layer):
    layer.selected_label -= 1


@register_label_action(
    trans._("Increase the currently selected label by one."),
)
def increase_label_id(layer):
    layer.selected_label += 1

```